## WinGO - A Wingsail Parameterisation and Optimisation Model

### Wingsail parameters dataset generation and thrust calculation

#### Wingsail parameters dataset generation

In this block, the code generates a setten amount of wingsail parameters with required constraints. The dataset should have 19 elements which define various wingsails.

In [ ]:
import numpy as np
from wingo import RawParameters

DATASET_DIR = './datasets/'
DATASET_SIZE = 4000

dataset_generator = RawParameters(dataset_size = DATASET_SIZE,
                          save_dir = DATASET_DIR,
                          data_trustworthiness = "positive",
                          save_csv = True)
raw_dataset_path = dataset_generator.generate()

print(f"Parameters dataset shape: {np.load(raw_dataset_path).shape}")

#### Wingsail thrust performance calculation

In this block, the code calculates the maximum thrust can provide by the wingsails in the dataset under different sailing conditions. In addition, the code reconstructs the dataset with calculation results into training and testing datasets.

In [ ]:
from wingo import DatasetBuild

PARAMETER_DATASET_PATH = raw_dataset_path
DATASET_CALCULATION_CONFIG = {'vessel_speed_range'   : [6, 10, 1],
                              'true_wind_speed_range': [6, 10, 1],
                              'true_wind_angle_range': [30, 150, 10],
                              'physical_constants'   : {'sea_surface_roughness'  : 2e-4,
                                                        'air_density'            : 1.205,
                                                        'air_kinematic_viscosity': 15.06e-6},
                              'aoa_range'            : [1, 25, 1],
                              'refinement_level'     : 1,
                              'neuralfoil_network'   : "xlarge"}

dataset_reconstructor = DatasetBuild(parameter_dataset_path = PARAMETER_DATASET_PATH,
                                     calculation_config = DATASET_CALCULATION_CONFIG)

calculated_dataset_path = dataset_reconstructor.thrust_calculation(save_csv = True)
print(f"Dataset with calculated thrust performance shape: {np.load(calculated_dataset_path).shape}")

In [ ]:
training_dataset_path, testing_dataset_path = dataset_reconstructor.build_training_and_testing_dataset(train_test_ratio  = 0.8,
                                                                                                       testing_filename  = 'ds2train.npy',
                                                                                                       training_filename = 'ds2test.npy')

print(f"Training dataset shape: {np.load(training_dataset_path).shape}")
print(f"Testing dataset shape: {np.load(testing_dataset_path).shape}")

### Thrust prediction regression model training and evaluation

##### Model setup

In this block, the code imports the essentail libararies and configs the parameters for the regression model training, evaluation and application.

In [2]:
# Import all libraries and modules for training
from wingo import DatasetPrepocessing, ThrustPredictor
from sklearn.metrics import r2_score
import numpy as np
import torch

# Activate bellow to run the section independently
training_dataset_path = './datasets/ds2train.npy'
testing_dataset_path  = './datasets/ds2test.npy'

# Pre-setup for training and apply the regression model
MODEL_DIR = './saved_model/'
NETWORK_NODES = 512
X_BOUNDS = np.array([[20  ,  100],  # Overall span range
                     [2   ,   30],  # Clearance over water range
                     [4   ,   25],  # Chord of bottom section range
                     [10  ,   25],  # Bottom section NACA profile range
                     [0.2 , 0.65],  # Distance to bottom from lower section range
                     [0.8 ,  1.1],  # Chord of lower section range
                     [8   ,   25],  # Lower section NACA profile range
                     [-0.2,  0.2],  # Lengthways offset of lower section range
                     [0.35, 0.85],  # Distance to bottom from middle section range
                     [0.6 ,  1.2],  # Chord of middle section range
                     [6   ,   25],  # Middle section NACA profile range
                     [-0.2,  0.4],  # Lengthways offset of middle section range
                     [0.8 , 0.95],  # Distance to bottom from upper section range
                     [0.3 ,    1],  # Chord of upper section range
                     [6   ,   25],  # Upper section NACA profile range
                     [0   ,  0.5],  # Lengthways offset of upper section range
                     [0.2 ,    1],  # Chord of tip section range
                     [6   ,   25],  # Tip section NACA profile range
                     [0   ,  0.7],  # Lengthways offset of tip section range
                     [6   ,   10],  # Vessel speed range
                     [6   ,   10],  # True wind speed range
                     [30  ,  150]]) # True wind angle range

training_dataset = np.load(training_dataset_path)
print(f"Training dataset shape: {training_dataset.shape}")
nodes = NETWORK_NODES
predictor_config  = {'model_label'      : 'Thrust_Regressor_V3',        # Label for the model
                     'input_dim'        : len(training_dataset[0]) - 1, # Dimension of the input feature X
                     'output_dim'       : 1,                            # Dimension of the target feature Y (the thrust to be predicted)
                     'latent_dim'       : nodes,                        # Dimension of the latent layer
                     'network_structure': [nodes,nodes,nodes,nodes],    # Structure of the neural network
                     'epochs_num'       : 1500,                         # Number of training epochs
                     'batch_size'       : 16384,                        # Batch size
                     'learning_rate'    : 0.0001,                       # Learning rate
                     'weight_decay'     : 0.01,                         # Weight decay
                     'vs_range'         : [6, 10, 1],                   # Vessel speed range
                     'tws_range'        : [6, 10, 1],                   # True wind speed range
                     'twa_range'        : [30, 150, 10]}                # True wind angle range

Training dataset shape: (1040000, 23)


##### Model training

In this block, the code instanciates the regression model and executes the training process, and returns a simplified evaluation result with the *validation dataset*.

In [ ]:
# Train the regression model for thrust prediction with the training dataset (1040000, 23)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

thrust_reg = ThrustPredictor(predictor_config, training_dataset, X_BOUNDS, training_device = 'cuda:0')

model_size = 0
for p in thrust_reg.model.parameters():
    model_size += p.numel()
print(f"Model size: {model_size}")

thrust_reg.train(16, 8)
thrust_reg.save(MODEL_DIR)

##### Model testing
In this block, the code loads the *testing dataset*, regression model and its configurations, to executes the testing process.

In [3]:
# Evaluate the regression model with the evaluation dataset (260000, 23)
testing_dataset = np.load(testing_dataset_path)
print(f"Evaluation dataset shape: {testing_dataset.shape}")

x_to_test, y_calc_shaped = DatasetPrepocessing(testing_dataset, X_BOUNDS).minmax_scaler()
thrust_reg = ThrustPredictor(predictor_config, training_dataset, X_BOUNDS, training_device = 'cuda:0')
thrust_reg.load(MODEL_DIR)
x_tensorised = torch.tensor(x_to_test, dtype=torch.float32).to(torch.device('cuda:0'))
y_predicted = thrust_reg.model(x_tensorised).to(torch.device('cpu')).detach().numpy()

print(f"R2 score of the predicted Y: {str(r2_score(y_calc_shaped, y_predicted))}")

Evaluation dataset shape: (260000, 23)
R2 score of the predicted Y: 0.9997756560311318


##### Extra verification with customised data

In this block, the wingsail parameters and sailing condisions can be customised. The thurst predicted by the regression model will be compared to the result calculated by the physical calculation method (xfoil method or neuralfoil method).

In [6]:
import time
from wingo import PreCalculation

# Define the customised wingsail and sail condition here
CUSTOMISED_WINGSAIL = [40, 10, 8, 15, 0.3, 1, 15, 0, 0.6, 1, 15, 0, 0.8, 0.9, 12, 0, 0.7, 10, 0]

# CUSTOMISED_WINGSAIL = [87.8500000, 29.2900000, 14.8900000, 19.0000000, 0.2100000, 0.8900000, 20.0000000, 0.1100000, 0.6600000, 0.8900000, 9.0000000, 0.2800000, 0.8500000, 0.6600000, 9.0000000, 0.4400000, 0.5400000, 9.0000000, 0.6100000]
SAILING_CONDITION = [8, 9, 110]

t = time.perf_counter()
wincal = PreCalculation(CUSTOMISED_WINGSAIL) # For Calculation class test
max_thrust, max_angle_of_attack = wincal.max_thrust_with_neuralfoil(SAILING_CONDITION)
# max_thrust, max_angle_of_attack = wincal.max_thrust_with_xfoil(SAILING_CONDITION)

print(f"Max thrust of {np.round(max_thrust, 3)} N found at angle of attack: {max_angle_of_attack} deg\n")
print(f"Thrust calculation takes {time.perf_counter() - t:8f} seconds!\n")


🚧 Notification: Input physical constants incomplete, reset to default values.

Max thrust of 26667.82 N found at angle of attack: 19 deg

Thrust calculation takes 0.263601 seconds!



In [20]:
t = time.perf_counter()
# Prepare the input features X
x_init = np.array(CUSTOMISED_WINGSAIL + SAILING_CONDITION + [999])[:, np.newaxis].T
x_input, _ = DatasetPrepocessing(x_init, X_BOUNDS).minmax_scaler()

# Predict the max thrust
thrust_reg = ThrustPredictor(predictor_config, x_init, X_BOUNDS, training_device = 'cuda:0')
thrust_reg.load(MODEL_DIR)
x_tensorised = torch.tensor(x_input).float().to(thrust_reg.device)
y_pred_log = thrust_reg.model(x_tensorised).to(torch.device('cpu')).detach().numpy()

print(f"Predicted raw y: {y_pred_log[0, 0]}")
print(f"Predicted max thrust: {10 ** y_pred_log[0, 0]} N")
print(f"Thrust prediction takes {time.perf_counter() - t} seconds!")

print(f"Delta: {abs(max_thrust - 10 ** y_pred_log[0, 0])/max_thrust * 100}%")

Predicted raw y: 4.4255690574646
Predicted max thrust: 26642.137015423723 N
Thrust prediction takes 0.03548250001040287 seconds!
Delta: 0.09630770064181457%
